In [1]:
import aocd
import dataclasses
import numpy as np
import enum

real_data = aocd.get_data(day=9, year=2022)
test_data = """R 4
U 4
L 3
D 1
R 4
D 1
L 5
R 2"""

In [2]:
from typing import Sequence, Union
import itertools


@enum.unique
class Direction(enum.Enum):
    """Defines the direction"""
    R = "Right"
    L = "Left"
    U = "Up"
    D = "Down"
    
class Map:
    """Defines the map instance."""
    
    def __init__(self):
        # Defines the size of the map [self.size, self.size] array
        self.size = 500
        self.tail_history = np.zeros([self.size, self.size])
        self.h_position = np.array([self.size // 2, self.size // 2])
        self.t_position = np.array([self.size // 2, self.size // 2])
        self.dir2vec = {
            Direction.R: np.array([0, 1]),
            Direction.L: np.array([0, -1]),
            Direction.D: np.array([1, 0]),
            Direction.U: np.array([-1, 0]),
        }
        
    def is_touching(self) -> bool:
        """Check whether or not H and T are touching."""
        # Relative position vector.
        vec = self.h_position - self.t_position
        return np.all(np.abs(vec) <= 1)
    
    def move_tail_accordingly(self) -> None:
        """Moves the tail according to the relative position."""
        # Relative position vector.
        vec = self.h_position - self.t_position
        
        if not self.is_touching():
            move_vec = np.clip(vec, -1, 1)
            self.t_position += move_vec.astype(int)
        
    def move_one(self, direction: Direction) -> None:
        """Moves the head and tail. One move only.
        
        Args:
            direction: the movement direction.
        """
        # Move head
        self.h_position += self.dir2vec[direction].astype(int)
        
        # Move tail accordinly
        self.move_tail_accordingly()
        
        self.log_tail()
        
        
    def move_multi(self, direction: Direction, n_steps: int) -> None:
        """Moves multiple steps.
        
        Args:
            direction: the movement direction.
            n_step: the number of steps.
        """
        for _ in range(n_steps):
            self.move_one(direction)
        
    def log_tail(self) -> None:
        """Logs the location of the tail onto `self.tail_history`."""
        a, b = self.t_position
        self.tail_history[a, b] = 1
        
    def get_total_track(self) -> int:
        """Computes the total tiles that tail has steped on."""
        return np.sum(self.tail_history).astype(int)

@dataclasses.dataclass
class SolverA:
    """
    A solver instance.
    
    args:
        raw_data: the raw input data.
    """
    raw_data: str

    def __post_init__(self):
        self.lines = self.raw_data.split("\n")
        # convert to numpy array
        direction_arr, n_steps_arr = [], []
        str2dir = {
            "U": Direction.U,
            "D": Direction.D,
            "L": Direction.L,
            "R": Direction.R
        }
        for line in self.lines:
            direction, n_steps = line.split(" ")
            direction_arr.append(str2dir[direction])
            n_steps_arr.append(int(n_steps))
        self.n_steps_arr = n_steps_arr
        self.direction_arr = direction_arr

    def find_answer(self) -> int:
        """Finds the answer.
        
        Returns:
            The answer.
        """
        my_map = Map()
        for direction, n_steps in zip(self.direction_arr, self.n_steps_arr):
            my_map.move_multi(direction, n_steps)
#         print(my_map.tail_history)
        return my_map.get_total_track()


In [3]:
SolverA(test_data).find_answer()

13

In [ ]:
answer = SolverA(real_data).find_answer()
aocd.submit(answer, part="a", day=9, year=2022)

In [42]:
from typing import Sequence, Union, List
import itertools
import copy

@enum.unique
class Direction(enum.Enum):
    """Defines the direction"""
    R = "Right"
    L = "Left"
    U = "Up"
    D = "Down"
     

def are_touching(vec: np.ndarray) -> bool:
    """Check whether or not the two given positions are `touching`.
    
    Args:
        vec: Relative position vector
    """
    return np.all(np.abs(vec) <= 1)
    
@dataclasses.dataclass
class Segment:
    """Describes 1 segment of the rope. A uses linked list data structure.
    
    Args:
        position: the position of the segment.
        tail: the tail segment.
    """
    position: np.ndarray
    tail: Union[List["Segment"], None]
        
    def __post_init__(self):
        self.dir2vec = {
            Direction.R: np.array([0, 1]),
            Direction.L: np.array([0, -1]),
            Direction.D: np.array([1, 0]),
            Direction.U: np.array([-1, 0]),
        }

        
    def move_tail_recursively(self) -> np.ndarray:
        """Moves the tail according to the relative position, recursively.
        
        Returns:
            tail position.
        """
        if self.tail is None:
            return self.position
        
        # Relative position vector.
        vec = self.position - self.tail.position
        if not are_touching(vec):
            move_vec = np.clip(vec, -1, 1)
            self.tail.position += move_vec.astype(int)

        return self.tail.move_tail_recursively()
        
        
    def move_one(self, direction: Direction) -> np.ndarray:
        """Moves the head and tail. One move only.
        
        Args:
            direction: the movement direction.
            
        Returns:
            position of the tail.
        """
        # Move head
        self.position += self.dir2vec[direction].astype(int)
        return self.move_tail_recursively()
        
        
    def move_multi(
        self, 
        direction: Direction, 
        n_steps: int, 
        tail_history: np.ndarray,
    ) -> np.ndarray:
        """Moves multiple steps.
        
        Args:
            direction: the movement direction.
            n_step: the number of steps.
            tail_history: the tiles the tail has been.
            
        Returns:
            The updated `tail_history`.
        """
        for _ in range(n_steps):
            tail_position = self.move_one(direction)
            a, b = tail_position
            tail_history[a, b] = 1
        return tail_history
  

@dataclasses.dataclass
class SolverB:
    """
    A solver instance.
    
    args:
        raw_data: the raw input data.
    """
    raw_data: str

    def __post_init__(self):
        self.lines = self.raw_data.split("\n")
        # convert to numpy array
        direction_arr, n_steps_arr = [], []
        str2dir = {
            "U": Direction.U,
            "D": Direction.D,
            "L": Direction.L,
            "R": Direction.R
        }
        for line in self.lines:
            direction, n_steps = line.split(" ")
            direction_arr.append(str2dir[direction])
            n_steps_arr.append(int(n_steps))
        self.n_steps_arr = n_steps_arr
        self.direction_arr = direction_arr

    def find_answer(self) -> int:
        """Finds the answer.
        
        Returns:
            The answer.
        """
        map_size = 500
        tail_history = np.zeros([map_size, map_size])
        init_position = np.array([map_size // 2, map_size // 2])
        
        rope = Segment(init_position, None)
        for _ in range(9):
            rope = Segment(copy.copy(init_position), rope)
        
        for direction, n_steps in zip(self.direction_arr, self.n_steps_arr):
            tail_history = rope.move_multi(direction, n_steps, tail_history)
        return np.sum(tail_history).astype(int)

In [43]:
import sys
import numpy
numpy.set_printoptions(threshold=sys.maxsize)

test_data = """R 5
U 8
L 8
D 3
R 17
D 10
L 25
U 20"""
SolverB(test_data).find_answer()

36

In [44]:
answer = SolverB(real_data).find_answer()
aocd.submit(answer, part="b", day=9, year=2022)

That's the right answer!  You are one gold star closer to collecting enough star fruit.You have completed Day 9! You can [Shareon
  Twitter
Mastodon] this victory or [Return to Your Advent Calendar].


<Response [200]>